In [1]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()

ws = Workspace.get(name='quick-starts-ws-125398',
                   subscription_id='ef22e2eb-a072-4db6-86df-7ea6643b1b5f',
                   resource_group='aml-quickstarts-125398',
                   )

experiment = Experiment(ws, 'myexperiment')

In [2]:
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print(compute.name, ":",compute.type)

ampl : ComputeInstance
cpu-cluster : AmlCompute


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# TOO Create TabularDataset using TabularDataFactory
# Data is located at:
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=url)

In [12]:
from train22 import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
# x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds) 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
train_data = x_train.join(y_train)

In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5)


In [14]:
# Submit your automl run

### YOUR CODE HERE ###
exp = Experiment(ws, 'automl')  
auto_run = exp.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_d8ded348-5656-4304-b5fd-4d60809949b8

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

WARNING - Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f30b2385550>, 'Connection to southcentralus.experiments.azureml.net timed out. (connect timeout=100)')': /history/v1.0/subscriptions/ef22e2eb-a072-4db6-86df-7ea6643b1b5f/resourceGroups/aml-quickstarts-125398/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-125398/experimentids/35e860d1-a9e4-4609-bee8-37ed689e01e5/runs/7eab38c4-6839-4c89-9e01-d58558bd62ee


Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RB389UBCP to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
StandardScalerWrapper XGBoostClassifier        0:00:43       0.9034    0.9136
        41   VotingEnsemble                                 0:02:04       0.9155    0.9155
        42   StackEnsemble                                  0:01:14       0.9135    0.9155
Stopping criteria reached at iteration 43. Ending experiment.
****************************************************************************************************
Current status: BestRunExplainModel. Best run model explanations started
Current status: ModelExplanationDataSetSetup. Model explanations data setup completed
Current status: PickSurrogateModel. Choosing LightGBM as the surrog